# EDA — Pedidos eB2B (100% PySpark)

Este cuaderno realiza un **Análisis Exploratorio de Datos (EDA)** usando **PySpark de principio a fin**.  
La lectura, limpieza, agregaciones y cálculos se hacen con Spark. Para los **gráficos**, se convierten **únicamente resultados agregados** y de tamaño pequeño a pandas/Matplotlib.

**Objetivo:** entender patrones de pedidos (especialmente el canal `DIGITAL`), montos de facturación y distribución por país, región, etc.


## 1. Configuración de entorno y sesión Spark

Se crea una sesión local de Spark utilizando todos los núcleos disponibles. Ajusta memoria/tuning si lo necesitas.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

spark = (
    SparkSession.builder
    .appName("next-digital-order-eb2b-eda")
    .master("local[*]")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .config("spark.driver.memory", "4g")
    .config("spark.sql.warehouse.dir", "./spark-warehouse")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

print("Spark version:", spark.version)

## 2. Carga de datos

Indica el directorio Parquet de tu dataset. Por defecto, se usa `dataset/dataset`.  
Si tu ruta es diferente, actualiza `DATA_DIR`.


In [ ]:
# === Parámetro de entrada ===
DATA_DIR = "dataset/dataset"  # Cambia esta ruta si tu dataset está en otra carpeta

# === Lectura ===
df = spark.read.parquet(DATA_DIR)

print("Número de filas:", df.count())
print("Número de columnas:", len(df.columns))
df.printSchema()

# Vista rápida
df.show(5, truncate=False)

## 3. Diccionario de datos (esperado)

| Columna                   | Tipo (esperado) | Descripción breve |
|--------------------------|-----------------|-------------------|
| `cliente_id`             | string          | Identificador de cliente |
| `pais_cd`                | string          | Código de país (GT, PE, EC, SV, etc.) |
| `region_comercial_txt`   | string          | Región comercial |
| `agencia_id`             | string          | Agencia |
| `ruta_id`                | string          | Ruta |
| `tipo_cliente_cd`        | string          | Tipo de cliente (TIENDA, MINIMARKET, etc.) |
| `madurez_digital_cd`     | string          | Nivel de madurez digital (ALTA, MEDIA, BAJA) |
| `estrellas_txt`          | string          | Puntuación/estrellas |
| `frecuencia_visitas_cd`  | string          | Frecuencia de visitas (ej. LMI, LMV, etc.) |
| `fecha_pedido_dt`        | timestamp       | Fecha/hora del pedido |
| `canal_pedido_cd`        | string          | Canal del pedido (DIGITAL, VENDEDOR, TELEFONO) |
| `facturacion_usd_val`    | double          | Monto facturado en USD |
| `materiales_distintos_val`| long           | Conteo de materiales distintos en el pedido |
| `cajas_fisicas`          | double          | Cajas físicas (volumen) |

> Si alguna columna no existe en tu dataset, las celdas siguientes la ignorarán automáticamente.


## 4. Calidad de datos

### 4.1 Nulos por columna
Se calcula el número de valores nulos por columna.


In [ ]:
# Conteo de nulos por columna
null_exprs = [F.sum(F.col(c).isNull().cast("int")).alias(c) for c in df.columns]
nulls = df.agg(*null_exprs)
nulls.show(truncate=False)

# Convierte a forma larga para facilitar visualización posterior
nulls_long = nulls.select(F.explode(F.array(*[F.struct(F.lit(c).alias("col"), F.col(c).alias("nulls")) for c in df.columns])).alias("x"))                   .select("x.col", "x.nulls")

nulls_long.createOrReplaceTempView("nulls_long")

### 4.2 Distintos por columna (muestra)

Para columnas categóricas, revisar recuento de valores distintos ayuda a detectar codificaciones extrañas.


In [ ]:
# Distintos aproximados por columna (puede ser costoso si son muchas columnas)
distinct_counts = df.agg(*[F.countDistinct(F.col(c)).alias(c) for c in df.columns])
distinct_counts.show(truncate=False)

### 4.3 Rango temporal

Se extraen **año y mes** de `fecha_pedido_dt` para conocer el rango y densidad temporal.


In [ ]:
df_dates = df.withColumn("year", F.year("fecha_pedido_dt"))               .withColumn("month", F.month("fecha_pedido_dt"))               .withColumn("year_month", F.date_format("fecha_pedido_dt", "yyyy-MM"))

df_dates.select(
    F.min("fecha_pedido_dt").alias("min_fecha"),
    F.max("fecha_pedido_dt").alias("max_fecha")
).show(truncate=False)

df_dates.groupBy("year_month").count().orderBy("year_month").show(12, truncate=False)

## 5. Distribuciones numéricas

Se exploran las variables numéricas clave (`facturacion_usd_val`, `cajas_fisicas`, `materiales_distintos_val`).  
Para graficar histogramas y boxplots se toman **muestras agregadas o limitadas** y se convierten a pandas.


In [ ]:
import matplotlib.pyplot as plt

def style_ax(ax):
    # Oculta ejes superior y derecho
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    return ax

def add_bar_labels(ax):
    for p in ax.patches:
        height = p.get_height()
        if height is not None:
            ax.annotate(f'{height:.0f}', (p.get_x() + p.get_width() / 2, height),
                        ha='center', va='bottom', fontsize=9, rotation=0)

num_cols = [c for c, t in df.dtypes if t in ("double", "float", "int", "bigint")]

print("Columnas numéricas detectadas:", num_cols)

# Histograma por variable (usando sample para no traer todo)
for col in ["facturacion_usd_val", "cajas_fisicas", "materiales_distintos_val"]:
    if col in df.columns:
        pdf = df.select(F.col(col).cast("double")).dropna().sample(fraction=0.05, seed=42).limit(100000).toPandas()
        if not pdf.empty:
            plt.figure(figsize=(6,4))
            ax = plt.gca()
            ax = style_ax(ax)
            ax.hist(pdf[col], bins=30)
            ax.set_title(f"Histograma — {col}")
            ax.set_xlabel(col)
            ax.set_ylabel("Frecuencia")
            plt.show()

## 6. Canal de pedido y adopción digital

Se analiza la participación del canal `DIGITAL` por país y su evolución temporal.


In [ ]:
# Participación por país
by_country = (
    df.groupBy("pais_cd")
      .pivot("canal_pedido_cd")
      .count()
      .fillna(0)
)

# Agrega columna con % digital
if "DIGITAL" in by_country.columns:
    by_country = by_country.withColumn("total", sum([F.col(c) for c in by_country.columns if c not in ("pais_cd")]))
    by_country = by_country.withColumn("pct_digital", (F.col("DIGITAL") / F.col("total")) * 100.0)

by_country.orderBy(F.desc("pct_digital")).show(truncate=False)

# Gráfico de % digital por país
if "pct_digital" in by_country.columns:
    pdf = by_country.select("pais_cd", F.round("pct_digital", 1).alias("pct_digital")).orderBy(F.desc("pct_digital")).toPandas()
    if not pdf.empty:
        plt.figure(figsize=(7,4))
        ax = plt.gca()
        ax = style_ax(ax)
        bars = ax.bar(pdf["pais_cd"], pdf["pct_digital"])
        add_bar_labels(ax)
        ax.set_title("% de pedidos DIGITAL por país")
        ax.set_xlabel("País")
        ax.set_ylabel("% DIGITAL")
        plt.tight_layout()
        plt.show()

In [ ]:
# Evolución mensual del % DIGITAL (time series)
df_m = df.withColumn("ym", F.date_format("fecha_pedido_dt", "yyyy-MM"))
monthly = df_m.groupBy("ym", "canal_pedido_cd").count()

# Pivot a wide format
monthly_pivot = monthly.groupBy("ym").pivot("canal_pedido_cd").sum("count").fillna(0)
cols = [c for c in monthly_pivot.columns if c not in ("ym")]
monthly_pivot = monthly_pivot.withColumn("total", sum([F.col(c) for c in cols]))
if "DIGITAL" in monthly_pivot.columns:
    monthly_pivot = monthly_pivot.withColumn("pct_digital", (F.col("DIGITAL")/F.col("total"))*100.0)

monthly_pivot.orderBy("ym").show(12, truncate=False)

# Gráfico línea % DIGITAL
if "pct_digital" in monthly_pivot.columns:
    pdf = monthly_pivot.select("ym", F.round("pct_digital", 1).alias("pct_digital")).orderBy("ym").toPandas()
    if not pdf.empty:
        plt.figure(figsize=(7,4))
        ax = plt.gca()
        ax = style_ax(ax)
        ax.plot(pdf["ym"], pdf["pct_digital"], marker="o")
        # etiquetas de datos
        for x, y in zip(pdf["ym"], pdf["pct_digital"]):
            ax.annotate(f"{y:.1f}", (x, y), textcoords="offset points", xytext=(0,5), ha="center", fontsize=8)
        ax.set_title("Evolución mensual del % DIGITAL")
        ax.set_xlabel("Año-Mes")
        ax.set_ylabel("% DIGITAL")
        plt.xticks(rotation=45, ha="right")
        plt.tight_layout()
        plt.show()

## 7. Facturación por país y región

Se examina la distribución de `facturacion_usd_val` por `pais_cd` y por `region_comercial_txt`.


In [ ]:
# Facturación total por país
if "facturacion_usd_val" in df.columns:
    rev_country = df.groupBy("pais_cd").agg(F.sum("facturacion_usd_val").alias("facturacion_usd_sum"))
    pdf = rev_country.orderBy(F.desc("facturacion_usd_sum")).toPandas()
    if not pdf.empty:
        plt.figure(figsize=(7,4))
        ax = plt.gca()
        ax = style_ax(ax)
        bars = ax.bar(pdf["pais_cd"], pdf["facturacion_usd_sum"].round(0))
        add_bar_labels(ax)
        ax.set_title("Facturación total por país (USD)")
        ax.set_xlabel("País")
        ax.set_ylabel("USD")
        plt.tight_layout()
        plt.show()

# Facturación por región (Top-N)
if set(["region_comercial_txt", "facturacion_usd_val"]).issubset(set(df.columns)):
    rev_region = df.groupBy("region_comercial_txt").agg(F.sum("facturacion_usd_val").alias("facturacion_usd_sum"))
    pdf = rev_region.orderBy(F.desc("facturacion_usd_sum")).limit(15).toPandas()
    if not pdf.empty:
        plt.figure(figsize=(8,4))
        ax = plt.gca()
        ax = style_ax(ax)
        bars = ax.bar(pdf["region_comercial_txt"], pdf["facturacion_usd_sum"].round(0))
        add_bar_labels(ax)
        ax.set_title("Top regiones por facturación (USD)")
        ax.set_xlabel("Región")
        ax.set_ylabel("USD")
        plt.xticks(rotation=45, ha="right")
        plt.tight_layout()
        plt.show()

## 8. Distribuciones por canal

Comparación de montos por `canal_pedido_cd`. Se muestran **boxplots** y/o estadísticas descriptivas.


In [ ]:
# Descriptivos por canal
if set(["canal_pedido_cd", "facturacion_usd_val"]).issubset(set(df.columns)):
    desc = (df.groupBy("canal_pedido_cd")
              .agg(F.count("*").alias("n"),
                   F.avg("facturacion_usd_val").alias("avg_fact"),
                   F.expr("percentile_approx(facturacion_usd_val, 0.5)").alias("p50_fact"),
                   F.max("facturacion_usd_val").alias("max_fact"))
           ).orderBy(F.desc("n"))
    desc.show(truncate=False)

    # Boxplot pequeño (muestra)
    pdf = (df.select("canal_pedido_cd", F.col("facturacion_usd_val").cast("double").alias("fact"))
             .dropna()
             .sample(fraction=0.05, seed=42)
             .limit(50000)
             .toPandas())
    if not pdf.empty:
        plt.figure(figsize=(7,4))
        ax = plt.gca()
        ax = style_ax(ax)
        ax.boxplot([pdf.loc[pdf["canal_pedido_cd"]==c, "fact"] for c in sorted(pdf["canal_pedido_cd"].unique())],
                   labels=sorted(pdf["canal_pedido_cd"].unique()), showmeans=True)
        # Etiquetas de datos: para boxplots no hay barras, así que anotamos medias
        grp = pdf.groupby("canal_pedido_cd")["fact"].mean().round(0)
        for i, (c, m) in enumerate(grp.items(), start=1):
            ax.annotate(f"avg={m:.0f}", (i, m), textcoords="offset points", xytext=(0,5), ha="center", fontsize=8)
        ax.set_title("Distribución de facturación por canal (muestra)")
        ax.set_xlabel("Canal")
        ax.set_ylabel("USD")
        plt.tight_layout()
        plt.show()

## 9. Correlaciones (numéricas)

Se calculan correlaciones de Pearson entre variables numéricas disponibles.


In [ ]:
num_cols = [c for c, t in df.dtypes if t in ("double", "float", "int", "bigint")]
num_cols = [c for c in num_cols if c != "year"]

if len(num_cols) >= 2:
    # Construir matriz de correlación en Spark (par de columnas)
    pairs = []
    for i in range(len(num_cols)):
        for j in range(i+1, len(num_cols)):
            c1, c2 = num_cols[i], num_cols[j]
            corr = df.select(c1, c2).corr(c1, c2)
            pairs.append((c1, c2, corr))

    # Mostrar top correlaciones absolutas
    pairs_sorted = sorted(pairs, key=lambda x: abs(x[2]) if x[2] is not None else -1, reverse=True)
    print("Top correlaciones (abs):")
    for c1, c2, r in pairs_sorted[:10]:
        print(f"{c1:>24} ~ {c2:<24} = {r:.3f}" if r is not None else f"{c1} ~ {c2} = None")

## 10. Chequeos rápidos de integridad

Duplicados obvios por `cliente_id` + fecha, y top clientes por facturación.


In [ ]:
# Duplicados por cliente-fecha
if set(["cliente_id", "fecha_pedido_dt"]).issubset(set(df.columns)):
    dups = (df.groupBy("cliente_id", "fecha_pedido_dt")
              .count().filter(F.col("count") > 1)
           )
    print("Duplicados cliente-fecha:", dups.count())
    dups.show(5, truncate=False)

# Top clientes por facturación acumulada
if set(["cliente_id", "facturacion_usd_val"]).issubset(set(df.columns)):
    top_clients = (df.groupBy("cliente_id")
                     .agg(F.sum("facturacion_usd_val").alias("fact_total"))
                   ).orderBy(F.desc("fact_total")).limit(20)
    top_clients.show(truncate=False)

## 11. Hallazgos iniciales y próximos pasos

- **Rango temporal:** revisa si hay periodos con vacíos de datos o picos inusuales de actividad.
- **Adopción digital:** observa países/regiones con mayor `pct_digital` para priorizar iniciativas.
- **Facturación:** identifica regiones/segmentos con mayor contribución en USD.
- **Distribuciones:** verifica outliers en `facturacion_usd_val` y `cajas_fisicas`.
- **Calidad:** columnas con muchos nulos pueden requerir imputación o exclusión según el modelado futuro.

**Siguientes pasos sugeridos (no implementados aquí):**
- Feature engineering para modelar probabilidad de próximo pedido `DIGITAL` (cohortes, recencia, frecuencia, valor — RFM).
- Estratificar por tipo de cliente y madurez digital.
- Preparar un set de entrenamiento a nivel cliente-mes con etiqueta `próximo canal = DIGITAL`.


In [ ]:
# Cierre ordenado de la sesión
spark.stop()
print("Spark session stopped.")